In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
from datetime import date
from tqdm import tqdm


class Scrap:
    def get_and_parse_url(self, url):
        try:
            result = requests.get(url)
            soup = BeautifulSoup(result.text, 'html.parser')
            stk_name = soup.find("h1", attrs={'class': 'pcstname'})
            stock_name = stk_name.text
            all_data = soup.find("div", attrs={'class': 'nsert'})
            m = all_data.text.split("\n")
            comm_sentiment = soup.find("div", attrs={'class': 'commounity_senti'})
            community_sentiment = comm_sentiment.text
            tech_rating = soup.find("div", attrs={'class': 'mt15 CTR pb20'})
            technical_rating = tech_rating.find('a')['title']
            valuation = soup.find_all("div", attrs={'class': 'value_txtfr'})
            p_e = valuation[1].text
            list_bsh = re.findall("parseInt\('(.*?)'\);", community_sentiment)
            
            buy_percentage = list_bsh[0]  if list_bsh[0] else None
            sell_percentage = list_bsh[1] if list_bsh[1] else None
            hold_percentage = list_bsh[2] if list_bsh[2] else None
            
            community_sentiment_data = [buy_percentage, sell_percentage, hold_percentage]
            while "" in m:
                m.remove("")
            data_of_one_url = self.make_dataframe(technical_rating, m, stock_name, community_sentiment_data, p_e)
            return data_of_one_url
        except:
            print(url)

    def make_dataframe(self, technical_rating, data, stock_name, community_sentiment_data, p_e):
        # Calling DataFrame constructor
        df = pd.DataFrame()
        try:
            df = df.append({"Stock_Name": stock_name,
                            "Price_Change": data[3],
                            "Prev_close": data[14],
                            "Open_Price": data[16],
                            "52_Week_Low": data[20],
                            "52_Week_High": data[22],
                            "Technical_Rating": technical_rating,
                            "buy_percentage": community_sentiment_data[0],
                            "sell_percentage": community_sentiment_data[1],
                            "hold_percentage": community_sentiment_data[2],
                            "p_e": p_e
                            }, ignore_index=True)
        except:
            print ("error found {}".format(df))
        return df

    def get_dynamic_url(self):
        url = "https://www.moneycontrol.com/india/stockpricequote/"
        list_of_url = []
        result = requests.get(url)
        soup = BeautifulSoup(result.text, 'html.parser')
        all_url = soup.find("table", attrs={"class": "pcq_tbl MT10"})
        for a in all_url.find_all('a', href=True):
            list_of_url.append(a['href'])
        self.get_category_for_list_of_url(list_of_url[:-1])

    def get_category_for_list_of_url(self, list_of_url):
        # TODO: Change all variable names, make it function specific
        categorization_dict = {}
        for i in list_of_url:
            category = i.split('/')[-3:]
            if categorization_dict.get(category[0]):
                categorization_dict[category[0]].append(i)
            else:
                categorization_dict[category[0]] = []
                categorization_dict[category[0]].append(i)
        self.get_data_for_each_url(categorization_dict)

    def get_data_for_each_url(self, categorization_dict):
        dfObj = pd.DataFrame(
            columns=['category', 'URL', 'stock_name', 'price_change', 'prev_close', 'open_price', 'fifty_two_week_low',
                     'fifty_two_week_high', 'Technical_Rating', "buy_percentage", "sell_percentage", "hold_percentage",
                     "Date"])
        for key, value in categorization_dict.items():
            url_list = value
            for url in url_list:
                data = self.get_and_parse_url(url)
                try:
                    dfObj = dfObj.append(
                        {"category": key, "URL": url, "stock_name": data["Stock_Name"].values[0],
                         "price_change": data["Price_Change"].values[0],
                         "prev_close": data["Prev_close"].values[0], "open_price": data["Open_Price"].values[0],
                         "fifty_two_week_low": data["52_Week_Low"].values[0],
                         "fifty_two_week_high": data["52_Week_High"].values[0],
                         "Technical_Rating": data["Technical_Rating"].values[0],
                         "buy_percentage": data["buy_percentage"].values[0],
                         "sell_percentage": data["sell_percentage"].values[0],
                         "hold_percentage": data["hold_percentage"].values[0],
                         "p_e":data["p_e"],
                         "Date": str(date.today())},
                        ignore_index=True)
                    break
                except Exception as e:
                    print(dfObj, e)
        dfObj.to_csv("money_control_data.csv")


scap_obj = Scrap()
scap_obj.get_dynamic_url()

https://www.moneycontrol.com/india/stockpricequote/diversified/3mindia/MI42
Empty DataFrame
Columns: [category, URL, stock_name, price_change, prev_close, open_price, fifty_two_week_low, fifty_two_week_high, Technical_Rating, buy_percentage, sell_percentage, hold_percentage, Date]
Index: [] 'NoneType' object is not subscriptable
https://www.moneycontrol.com/india/stockpricequote/miscellaneous/arvindfashionslimited/ARVIN54248
                       category  \
0                   diversified   
1                financegeneral   
2            financeinvestments   
3               pharmaceuticals   
4                   cementmajor   
5                       trading   
6         infrastructuregeneral   
7   powergenerationdistribution   
8    powertransmissionequipment   
9                        retail   
10           transportlogistics   

                                                  URL  \
0   https://www.moneycontrol.com/india/stockpriceq...   
1   https://www.moneycontrol.com/ind

https://www.moneycontrol.com/india/stockpricequote/domesticappliances/bajajelectricals/BE
                        category  \
0                    diversified   
1                 financegeneral   
2             financeinvestments   
3                pharmaceuticals   
4                    cementmajor   
5                        trading   
6          infrastructuregeneral   
7    powergenerationdistribution   
8     powertransmissionequipment   
9                         retail   
10            transportlogistics   
11                 miscellaneous   
12              castingsforgings   
13               paintsvarnishes   
14                     chemicals   
15               autoancillaries   
16               steeltubespipes   
17      hospitalsmedicalservices   
18                         tyres   
19                  autolcvshcvs   
20  constructioncontractingcivil   
21                      plastics   
22            banksprivatesector   
23                   aquaculture   
24        

https://www.moneycontrol.com/india/stockpricequote/domesticappliances/hawkinscooker/HC02
                        category  \
0                    diversified   
1                 financegeneral   
2             financeinvestments   
3                pharmaceuticals   
4                    cementmajor   
5                        trading   
6          infrastructuregeneral   
7    powergenerationdistribution   
8     powertransmissionequipment   
9                         retail   
10            transportlogistics   
11                 miscellaneous   
12              castingsforgings   
13               paintsvarnishes   
14                     chemicals   
15               autoancillaries   
16               steeltubespipes   
17      hospitalsmedicalservices   
18                         tyres   
19                  autolcvshcvs   
20  constructioncontractingcivil   
21                      plastics   
22            banksprivatesector   
23                   aquaculture   
24         

https://www.moneycontrol.com/india/stockpricequote/domesticappliances/ttkprestige/TTK02
                        category  \
0                    diversified   
1                 financegeneral   
2             financeinvestments   
3                pharmaceuticals   
4                    cementmajor   
5                        trading   
6          infrastructuregeneral   
7    powergenerationdistribution   
8     powertransmissionequipment   
9                         retail   
10            transportlogistics   
11                 miscellaneous   
12              castingsforgings   
13               paintsvarnishes   
14                     chemicals   
15               autoancillaries   
16               steeltubespipes   
17      hospitalsmedicalservices   
18                         tyres   
19                  autolcvshcvs   
20  constructioncontractingcivil   
21                      plastics   
22            banksprivatesector   
23                   aquaculture   
24          

https://www.moneycontrol.com/india/stockpricequote/abrasives/carborundumuniversal/CU
                             category  \
0                         diversified   
1                      financegeneral   
2                  financeinvestments   
3                     pharmaceuticals   
4                         cementmajor   
5                             trading   
6               infrastructuregeneral   
7         powergenerationdistribution   
8          powertransmissionequipment   
9                              retail   
10                 transportlogistics   
11                      miscellaneous   
12                   castingsforgings   
13                    paintsvarnishes   
14                          chemicals   
15                    autoancillaries   
16                    steeltubespipes   
17           hospitalsmedicalservices   
18                              tyres   
19                       autolcvshcvs   
20       constructioncontractingcivil   
21           

...

https://www.moneycontrol.com/india/stockpricequote/textilesmachinery/lakshmimachineworks/LMW
                          category  \
0                      diversified   
1                   financegeneral   
2               financeinvestments   
3                  pharmaceuticals   
4                      cementmajor   
5                          trading   
6            infrastructuregeneral   
7      powergenerationdistribution   
8       powertransmissionequipment   
9                           retail   
10              transportlogistics   
11                   miscellaneous   
12                castingsforgings   
13                 paintsvarnishes   
14                       chemicals   
15                 autoancillaries   
16                 steeltubespipes   
17        hospitalsmedicalservices   
18                           tyres   
19                    autolcvshcvs   
20    constructioncontractingcivil   
21                        plastics   
22              banksprivatesecto

https://www.moneycontrol.com/india/stockpricequote/cementproductsbuildingmaterials/orientrefractories/OR01
                          category  \
0                      diversified   
1                   financegeneral   
2               financeinvestments   
3                  pharmaceuticals   
4                      cementmajor   
5                          trading   
6            infrastructuregeneral   
7      powergenerationdistribution   
8       powertransmissionequipment   
9                           retail   
10              transportlogistics   
11                   miscellaneous   
12                castingsforgings   
13                 paintsvarnishes   
14                       chemicals   
15                 autoancillaries   
16                 steeltubespipes   
17        hospitalsmedicalservices   
18                           tyres   
19                    autolcvshcvs   
20    constructioncontractingcivil   
21                        plastics   
22              ban

https://www.moneycontrol.com/india/stockpricequote/bearings/schaefflerindia/FAG
                                 category  \
0                             diversified   
1                          financegeneral   
2                      financeinvestments   
3                         pharmaceuticals   
4                             cementmajor   
5                                 trading   
6                   infrastructuregeneral   
7             powergenerationdistribution   
8              powertransmissionequipment   
9                                  retail   
10                     transportlogistics   
11                          miscellaneous   
12                       castingsforgings   
13                        paintsvarnishes   
14                              chemicals   
15                        autoancillaries   
16                        steeltubespipes   
17               hospitalsmedicalservices   
18                                  tyres   
19                  

https://www.moneycontrol.com/india/stockpricequote/bearings/skfindia/SKF01
                                 category  \
0                             diversified   
1                          financegeneral   
2                      financeinvestments   
3                         pharmaceuticals   
4                             cementmajor   
5                                 trading   
6                   infrastructuregeneral   
7             powergenerationdistribution   
8              powertransmissionequipment   
9                                  retail   
10                     transportlogistics   
11                          miscellaneous   
12                       castingsforgings   
13                        paintsvarnishes   
14                              chemicals   
15                        autoancillaries   
16                        steeltubespipes   
17               hospitalsmedicalservices   
18                                  tyres   
19                       


                                 category  \
0                             diversified   
1                          financegeneral   
2                      financeinvestments   
3                         pharmaceuticals   
4                             cementmajor   
5                                 trading   
6                   infrastructuregeneral   
7             powergenerationdistribution   
8              powertransmissionequipment   
9                                  retail   
10                     transportlogistics   
11                          miscellaneous   
12                       castingsforgings   
13                        paintsvarnishes   
14                              chemicals   
15                        autoancillaries   
16                        steeltubespipes   
17               hospitalsmedicalservices   
18                                  tyres   
19                           autolcvshcvs   
20           constructioncontractingcivil   
21       